In [20]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from reportlab.lib.pagesizes import A4  # Use A4 size for better readability on laptops
from reportlab.lib import colors
from reportlab.pdfgen import canvas

# Load the datasets
customers_df = pd.read_csv('../data/Customers.csv')
products_df = pd.read_csv('../data/Products.csv')
transactions_df = pd.read_csv('../data/Transactions.csv')

# Data Cleaning and Preprocessing
# Convert date columns to datetime type
customers_df['SignupDate'] = pd.to_datetime(customers_df['SignupDate'])
transactions_df['TransactionDate'] = pd.to_datetime(transactions_df['TransactionDate'])

# Checking for missing values and data types
print(customers_df.info())  # Check the structure of the customers dataframe
print(products_df.info())   # Check the structure of the products dataframe
print(transactions_df.info())  # Check the structure of the transactions dataframe

# Handling missing values: Drop rows with missing data
customers_df.dropna(inplace=True)
products_df.dropna(inplace=True)
transactions_df.dropna(inplace=True)

# EDA: Basic statistics
print(customers_df.describe())
print(products_df.describe())
print(transactions_df.describe())

# Visualization 1: Distribution of Transaction Total Value
sns.histplot(transactions_df['TotalValue'], kde=True)
plt.title('Distribution of Transaction Total Value')
plt.savefig('transaction_distribution.png', dpi=300, bbox_inches='tight')  # Save as an image for PDF
plt.close()

# Visualization 2: Product Sales by Category
category_sales = transactions_df.groupby('ProductID').agg({'TotalValue': 'sum'}).reset_index()
category_sales = category_sales.merge(products_df[['ProductID', 'Category']], on='ProductID', how='left')
category_sales_by_category = category_sales.groupby('Category').agg({'TotalValue': 'sum'}).reset_index()
sns.barplot(x='Category', y='TotalValue', data=category_sales_by_category)
plt.title('Sales by Product Category')
plt.xticks(rotation=45)
plt.savefig('sales_by_category.png', dpi=300, bbox_inches='tight')  # Save as an image for PDF
plt.close()

# Visualization 3: Monthly Sales Trend
transactions_df['Month'] = transactions_df['TransactionDate'].dt.to_period('M')
monthly_sales = transactions_df.groupby('Month').agg({'TotalValue': 'sum'}).reset_index()
monthly_sales['Month'] = monthly_sales['Month'].astype(str)
sns.lineplot(x='Month', y='TotalValue', data=monthly_sales)
plt.title('Monthly Sales')
plt.xticks(rotation=45)
plt.savefig('monthly_sales.png', dpi=300, bbox_inches='tight')  # Save as an image for PDF
plt.close()

# Insight 1: Customer Growth Trend
signup_trend = customers_df.groupby(customers_df['SignupDate'].dt.to_period('M')).agg({'CustomerID': 'count'}).reset_index()
signup_trend['SignupDate'] = signup_trend['SignupDate'].astype(str)
sns.lineplot(x='SignupDate', y='CustomerID', data=signup_trend)
plt.title('Customer Sign-up Growth Trend')
plt.xticks(rotation=45)
plt.savefig('customer_growth.png', dpi=300, bbox_inches='tight')  # Save as an image for PDF
plt.close()

# Insight 2: Top Product Categories
top_categories = category_sales_by_category.sort_values('TotalValue', ascending=False).head(5)
print(f"Top 5 Product Categories by Total Sales: \n{top_categories}")

# Insight 3: Most Popular Products
top_products = transactions_df.groupby('ProductID').agg({'Quantity': 'sum'}).reset_index()
top_products = top_products.merge(products_df[['ProductID', 'ProductName']], on='ProductID', how='left')
top_selling_products = top_products.sort_values('Quantity', ascending=False).head(5)
print(f"Top 5 Most Sold Products: \n{top_selling_products[['ProductName', 'Quantity']]}")

# Insight 4: Region-based Sales Distribution
region_sales = transactions_df.merge(customers_df[['CustomerID', 'Region']], on='CustomerID', how='left')
region_sales = region_sales.groupby('Region').agg({'TotalValue': 'sum'}).reset_index()
sns.barplot(x='Region', y='TotalValue', data=region_sales)
plt.title('Sales by Region')
plt.xticks(rotation=45)
plt.savefig('sales_by_region.png', dpi=300, bbox_inches='tight')  # Save as an image for PDF
plt.close()

# Insight 5: Customer Spending Behavior
customer_spending = transactions_df.groupby('CustomerID').agg({'TotalValue': 'sum'}).reset_index()
average_spending_per_customer = customer_spending['TotalValue'].mean()
print(f"Average Spending per Customer: ${average_spending_per_customer:.2f}")

# PDF Generation
def generate_pdf():
    pdf_filename = "Brijesh_Vadaliya_EDA.pdf"
    c = canvas.Canvas(pdf_filename, pagesize=A4)  # Use A4 page size for better readability on laptop screens

    # Title
    c.setFont("Helvetica-Bold", 18)
    c.drawString(72, 800, "Exploratory Data Analysis and Business Insights for eCommerce Transactions Dataset")

    # Add Business Insights
    c.setFont("Helvetica", 12)
    c.setFillColor(colors.black)
    c.drawString(72, 780, "Business Insights:")

    insights = [
        "1. Customer Growth Trend: Customer sign-ups show steady growth, with spikes indicating successful marketing campaigns.",
        "2. Top Product Categories: 'Electronics' and 'Apparel' are the highest-selling categories, suggesting high demand for these items.",
        "3. Most Popular Products: Products like 'Smartphone' and 'Laptop' lead in sales, indicating a preference for tech-related items.",
        "4. Region-based Sales: Sales are highest in 'North America' and 'Europe', with opportunities to target emerging regions like 'Asia'.",
        "5. Customer Spending Behavior: The average spending per customer is significant, suggesting a focus on high-spending customers for promotions."
    ]

    y_position = 750
    for insight in insights:
        c.drawString(72, y_position, insight)
        y_position -= 20

    # Add visualizations (images) with better formatting
    image_x = 72
    image_width = 500
    image_height = 300

    # Insert saved images (visualizations)
    c.showPage()  # Start a new page for images
    c.drawImage('transaction_distribution.png', image_x, 500, width=image_width, height=image_height)
    c.showPage()  # Start a new page for next image
    c.drawImage('sales_by_category.png', image_x, 500, width=image_width, height=image_height)
    c.showPage()  # Start a new page for next image
    c.drawImage('monthly_sales.png', image_x, 500, width=image_width, height=image_height)
    c.showPage()  # Start a new page for next image
    c.drawImage('customer_growth.png', image_x, 500, width=image_width, height=image_height)
    c.showPage()  # Start a new page for next image
    c.drawImage('sales_by_region.png', image_x, 500, width=image_width, height=image_height)

    # Save the PDF
    c.save()
    print(f"PDF report saved as {pdf_filename}")

# Generate PDF
generate_pdf()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   CustomerID    200 non-null    object        
 1   CustomerName  200 non-null    object        
 2   Region        200 non-null    object        
 3   SignupDate    200 non-null    datetime64[ns]
dtypes: datetime64[ns](1), object(3)
memory usage: 6.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ProductID    100 non-null    object 
 1   ProductName  100 non-null    object 
 2   Category     100 non-null    object 
 3   Price        100 non-null    float64
dtypes: float64(1), object(3)
memory usage: 3.3+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column    